In [65]:
import requests
from bs4 import BeautifulSoup 
import csv
import pandas as pd

In [74]:
post_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(post_url)
pages = BeautifulSoup(page.content,"html.parser")

In [86]:
! rm file.csv && touch file.csv && chmod 777 file.csv
values = pages.find('tbody').findAll("tr")
for vals in values:
    vals = vals.find_all('td')
    with open('file.csv', 'a') as csv_file:
        writer = csv.writer(csv_file)
        if len(vals)==3:
            postcode = vals[0].text.strip()
            borough = vals[1].text.strip()
            neighbourhood = vals[2].text.strip()
            writer.writerow([postcode,borough,neighbourhood])
        else:
            writer.writerow(['Postcode','Borough','Neighbourhood'])
        

In [100]:
df = pd.read_csv('file.csv')

In [101]:
df = df[~df['Borough'].isin(['Not assigned'])]

In [102]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [115]:
df['Neighbourhood'] = df['Borough'].where(df['Neighbourhood']=='Not assigned',df['Neighbourhood'])
df

In [116]:
df.shape

(103, 3)

In [114]:
geocode = pd.read_csv('Geospatial_Coordinates.csv')
geocode.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
df1 = df.merge(geocode, how='left', on="Postcode" )
df1.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
